# Big-5 Personality Classifier Training

This notebook trains a dual-embedding (RoBERTa + Sentence-BERT) model to classify personality traits from text.

**Dataset:** essays-big5 from Hugging Face

**Target:** Binary classification for each Big-5 trait (0=low, 1=high)
- **E**xtraversion
- **N**euroticism
- **A**greeableness
- **C**onscientiousness
- **O**penness

**Runtime:** Select **GPU** for faster training (Runtime → Change runtime type → T4 GPU)

## 1. Install Dependencies

In [ ]:
!pip install -q transformers sentence-transformers datasets scikit-learn torch

## 2. Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import numpy as np

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 3. Data Processor

Loads the essays-big5 dataset and prepares it for training.

In [ ]:
class DataProcessor:
    """
    Processes the essays-big5 dataset for personality classification.
    
    Binary labels (0=low trait, 1=high trait) for multi-label classification.
    Each person can be high (1) or low (0) on each trait independently.
    """
    
    def __init__(self):
        self.trait_names = ['Extraversion', 'Neuroticism', 'Agreeableness', 
                           'Conscientiousness', 'Openness']

    def load_and_preprocess(self):
        """
        Load and preprocess the essays-big5 dataset from Hugging Face.
        
        Returns:
            X_train, X_test: Essay texts
            y_train, y_test: Big-5 personality scores (binary: 0 or 1)
                             Shape: (n_samples, 5) for [E, N, A, C, O]
        """
        print("Loading dataset from Hugging Face...")
        ds = load_dataset("jingjietan/essays-big5", split="train")
        texts = ds["text"]
        
        # Extract Big-5 binary labels in OCEAN order
        # Values are binary: 0 (low on trait) or 1 (high on trait)
        big5_scores = np.array([
            [float(row["E"]), float(row["N"]), float(row["A"]), 
             float(row["C"]), float(row["O"])]
            for row in ds
        ])
        
        print(f"Dataset size: {len(texts)} essays")
        print(f"Label distribution (% high on each trait):")
        for i, trait in enumerate(self.trait_names):
            pct = np.mean(big5_scores[:, i]) * 100
            print(f"  {trait}: {pct:.1f}%")
        
        # Split into train/test sets (80/20)
        X_train, X_test, y_train, y_test = train_test_split(
            texts, big5_scores, test_size=0.2, random_state=42
        )
        
        return X_train, X_test, y_train, y_test

## 4. Model Architecture

Dual-embedding model combining RoBERTa and Sentence-BERT.

In [ ]:
class PersonalityClassifier(nn.Module):
    """
    Dual-embedding personality classifier for Big-5 trait prediction.
    
    - RoBERTa: Captures token-level context and linguistic patterns
    - Sentence-BERT: Captures semantic meaning and overall coherence
    - Frozen base models to prevent overfitting
    - Output: 5 binary predictions for [E, N, A, C, O]
    """
    
    def __init__(self, model_name='roberta-base'):
        super(PersonalityClassifier, self).__init__()
        self.roberta = RobertaModel.from_pretrained(model_name)
        self.sbert = SentenceTransformer('all-mpnet-base-v2')
        
        # Freeze RoBERTa parameters to leverage pre-trained knowledge
        for param in self.roberta.parameters():
            param.requires_grad = False
            
        # Combined embedding dimension (RoBERTa + sBERT)
        roberta_dim = 768
        sbert_dim = 768
        combined_dim = roberta_dim + sbert_dim
        
        # Classifier layers for Big-5 personality prediction
        self.classifier = nn.Sequential(
            nn.Linear(combined_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 5)  # Output: 5 Big-5 traits [E, N, A, C, O]
        )
        
    def forward(self, input_ids, attention_mask, texts):
        # Get RoBERTa embeddings using [CLS] token
        roberta_outputs = self.roberta(input_ids, attention_mask=attention_mask)
        roberta_embeddings = roberta_outputs.last_hidden_state[:, 0, :]
        
        # Get sBERT embeddings
        sbert_embeddings = torch.tensor(self.sbert.encode(texts)).to(input_ids.device)
        
        # Concatenate embeddings
        combined_embeddings = torch.cat([roberta_embeddings, sbert_embeddings], dim=1)
        
        # Pass through classifier to predict Big-5 traits
        big5_predictions = self.classifier(combined_embeddings)
        
        return big5_predictions

## 5. Dataset Class

In [ ]:
class TextPersonalityDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.float32)
        }

## 6. Training Function

In [ ]:
def train_model(model, train_loader, val_loader, device, num_epochs=10):
    """
    Train the personality classifier with comprehensive metrics.
    """
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.05)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1)
    
    best_val_loss = float('inf')
    patience = 2
    wait = 0
    trait_names = ['Extraversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness']
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            texts = batch['text']
            
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask, texts)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_train_loss = total_loss / len(train_loader)
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_preds = []
        val_targets = []
        
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                texts = batch['text']
                
                outputs = model(input_ids, attention_mask, texts)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                val_preds.append(outputs.cpu().numpy())
                val_targets.append(labels.cpu().numpy())
        
        avg_val_loss = val_loss / len(val_loader)
        
        # Calculate detailed metrics
        val_preds = np.concatenate(val_preds, axis=0)
        val_targets = np.concatenate(val_targets, axis=0)
        
        # Overall MAE (Mean Absolute Error)
        overall_mae = np.mean(np.abs(val_preds - val_targets))
        
        # Binary classification accuracy (threshold at 0.5)
        binary_accuracy = np.mean((val_preds > 0.5).astype(int) == val_targets.astype(int)) * 100
        
        # Per-trait binary accuracy
        per_trait_accuracy = np.mean((val_preds > 0.5).astype(int) == val_targets.astype(int), axis=0) * 100
        
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        print(f"  Val MAE: {overall_mae:.4f} | Binary Accuracy: {binary_accuracy:.2f}%")
        print(f"  Per-trait Accuracy: ", end="")
        for i, (name, acc) in enumerate(zip(trait_names, per_trait_accuracy)):
            print(f"{name[:3]}={acc:.1f}%", end=" " if i < 4 else "\n")
        
        scheduler.step(avg_val_loss)
        
        # Save checkpoints at specific epochs for analysis
        if epoch+1 in [3, 4]:
            torch.save(model.state_dict(), f'weights_epoch_{epoch+1}.pt')
            print(f"  Saved checkpoint: weights_epoch_{epoch+1}.pt")
        
        # Early stopping with best model saving
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pt')
            print(f"  ✓ New best model saved (val_loss: {best_val_loss:.4f})")
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print(f"\nEarly stopping at epoch {epoch+1} (no improvement for {patience} epochs)")
                break
        
        print()  # Blank line for readability

## 7. Main Training Pipeline

**Note:** This will take approximately:
- **With GPU (T4)**: ~15-20 minutes
- **Without GPU (CPU)**: ~2-3 hours

Make sure to enable GPU runtime for faster training!

In [ ]:
def main():
    print("="*60)
    print("Big-5 Personality Binary Classifier Training")
    print("="*60)
    
    # Initialize data processor
    print("\n[1/5] Loading and preprocessing data...")
    data_processor = DataProcessor()
    X_train, X_test, y_train, y_test = data_processor.load_and_preprocess()
    print(f"  Dataset: {len(X_train)} training samples, {len(X_test)} test samples")
    print(f"  Target: Binary Big-5 traits (0=low, 1=high) [E, N, A, C, O]")
    print(f"  Task: Multi-label binary classification")
    
    # Initialize tokenizer and model
    print("\n[2/5] Initializing model...")
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    model = PersonalityClassifier()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Count trainable parameters
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"  Device: {device}")
    print(f"  Trainable parameters: {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.1f}%)")
    
    # Create datasets and dataloaders
    print("\n[3/5] Creating data loaders...")
    train_dataset = TextPersonalityDataset(X_train, y_train, tokenizer)
    test_dataset = TextPersonalityDataset(X_test, y_test, tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16)
    print(f"  Batch size: 16")
    print(f"  Training batches: {len(train_loader)}")
    print(f"  Validation batches: {len(test_loader)}")
    
    # Train the model
    print("\n[4/5] Training model...")
    print("-"*60)
    train_model(model, train_loader, test_loader, device)
    
    print("\n[5/5] Training complete!")
    print("  Best model saved to: best_model.pt")
    print("="*60)

# Run the training
main()

## 8. Download Trained Model (Optional)

Download the trained model to your local machine.

In [ ]:
from google.colab import files

# Download the best model
files.download('best_model.pt')

# Optionally download epoch checkpoints
# files.download('weights_epoch_3.pt')
# files.download('weights_epoch_4.pt')

## 9. Test the Model (Optional)

Make predictions on custom text samples.

In [ ]:
def predict_personality(text, model, tokenizer, device):
    """
    Predict personality traits from text.
    """
    model.eval()
    
    # Tokenize
    encoding = tokenizer(
        text,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Predict
    with torch.no_grad():
        outputs = model(input_ids, attention_mask, [text])
        predictions = (outputs.cpu().numpy() > 0.5).astype(int)[0]
        scores = outputs.cpu().numpy()[0]
    
    # Display results
    trait_names = ['Extraversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness']
    print("\nPredicted Personality Profile:")
    print("-" * 50)
    for trait, pred, score in zip(trait_names, predictions, scores):
        level = "HIGH" if pred == 1 else "LOW"
        print(f"{trait:18} : {level:4} (score: {score:.3f})")

# Example usage
sample_text = """I absolutely love meeting new people and going to parties. 
I'm always the center of attention and I thrive in social situations. 
I get energized by being around others and hate being alone."""

predict_personality(sample_text, model, tokenizer, device)